<a href="https://colab.research.google.com/github/Naeima/fooKG/blob/main/foo_KG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# @title
!pip install rdflib
!pip install requests
!pip install SPARQLWrapper
!pip install rdflib spacy
!pip python -m spacy download en_core_web_sm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.1/565.1 kB 9.3 MB/s eta 0:00:00
ERROR: unknown command "python"


In [4]:
# @title
import io
import os
import rdflib
from rdflib import URIRef, Literal, Graph, RDF, Namespace, plugin
from rdflib.plugins import sparql
from rdflib.plugins.sparql import prepareQuery
from rdflib.parser import Parser
from rdflib import Graph
from rdflib.plugins.sparql import prepareQuery
from SPARQLWrapper import SPARQLWrapper, JSON, N3
import requests
import json
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON

In [23]:
# @title
# Geo-Filtered RDF Extractor with Folium Map + Export (19/07/2025)
from rdflib import Graph
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, shape
import folium
from folium.plugins import Draw
import ipywidgets as widgets
from IPython.display import display, clear_output
from google.colab import files
import json

# Step 1: Load RDF graph
g = Graph()
g.parse("https://naeima.github.io/fooKG/index.ttl", format="ttl")

# Step 2: Extract lat/long + metadata into flat dicts
records = {}
for s, p, o in g:
    s_str = str(s)
    p_str = str(p).lower()
    if s_str not in records:
        records[s_str] = {"subject": s_str}
    if 'latitude' in p_str:
        try:
            records[s_str]["latitude"] = float(o)
        except:
            continue
    elif 'longitude' in p_str:
        try:
            records[s_str]["longitude"] = float(o)
        except:
            continue
    else:
        pred = p_str.split("/")[-1] if "/" in p_str else p_str
        records[s_str][pred] = str(o)

# Step 3: Convert to GeoDataFrame
filtered = [v for v in records.values() if "latitude" in v and "longitude" in v]
df = pd.DataFrame(filtered)
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude), crs="EPSG:4326")

# Step 4: Create interactive folium map
m = folium.Map(location=[gdf.latitude.mean(), gdf.longitude.mean()], zoom_start=12)
draw = Draw(export=True, filename="polygon.geojson", draw_options={'polygon': True, 'rectangle': True})
draw.add_to(m)

# Step 5: Add all RDF points to the map
for _, row in gdf.iterrows():
    tooltip = "<br>".join([f"{k}: {v}" for k, v in row.items() if k not in ("geometry", "latitude", "longitude")])
    folium.Marker(
        location=[row.latitude, row.longitude],
        tooltip=tooltip,
        popup=row.get("id", "")
    ).add_to(m)

# Display the map
display(m)

# Step 6: Upload exported polygon and filter
upload = widgets.FileUpload(accept='.geojson', multiple=False)
filter_button = widgets.Button(description="Filter Points")
output = widgets.Output()

def filter_points(_):
    clear_output()
    if not upload.value:
        print("Please upload a GeoJSON polygon first.")
        return

    content = next(iter(upload.value.values()))
    geojson = json.loads(content['content'].decode())
    poly_geom = shape(geojson['features'][0]['geometry'])

    filtered_gdf = gdf[gdf.geometry.within(poly_geom)]

    if filtered_gdf.empty:
        print("No points found inside the selected area.")
    else:
        # Save and download as GeoJSON
        path = "filtered_jasmin.geojson"
        filtered_gdf.to_file(path, driver="GeoJSON")
        files.download(path)

filter_button.on_click(filter_points)

# Step 7: Display widgets
display(widgets.VBox([upload, filter_button, output]))
